#Морфологический анализ текста на русском языке
##Библиотека pymorphy2
Pymorphy2 — морфологический процессор с открытым исходным
кодом, предоставляет все функции полного морфологического анализа и
синтеза словоформ. Процессор базируется на словарной морфологии и использует словарные данные проекта **OpenCorpora**


Установка

In [10]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)


Словари распространяются отдельными пакетами и требуют периодически обновлений

In [11]:
!pip install -U pymorphy2-dicts-ru

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import pymorphy2

В pymorphy2 для морфологического анализа слов есть класс ``` MorphAnalyzer ``` (по умолчанию стоит русский язык)

In [13]:
morph = pymorphy2.MorphAnalyzer()

С помощью метода `MorphAnalyzer.parse()` можно разобрать отдельное слово. Метод возвращает один или несколько объектов типа `Parse` с информацией о том, как слово может быть разобрано. Ниже проведен разбор слова 'стали'.

In [14]:
morph.parse('стали')

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.975342, methods_stack=((DictionaryAnalyzer(), 'стали', 945, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.010958, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.005479, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 9),))]

Структура ответа состоит из поля:
- word – исходное слово;
- tag – грамматические характеристики; 
  *   например **OpencorporaTag('VERB,perf,intr plur,past,indc')** дает следующую информацию: слово - глагол (VERB) совершенного вида (perf), непереходный (intr), множественного числа (plur), прошедшего времени (past), изъявительного наклонения (indc). [Обозначения для граммем.](https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#grammeme-docs)
- normal_form – начальная форма слова; 
- score – это оценка $P(tag|word)$ вероятности того, что данный разбор правильный.


###Выбор правильного разбора
pymorphy2 возвращает все допустимые варианты разбора, но на практике обычно нужен только один вариант, правильный. Для этого у разбора есть параметр score. Условная вероятность $P(tag|word)$ оценивается на основе корпуса **OpenCorpora**: ищутся все неоднозначные слова со снятой неоднозначностью, для каждого слова считается, сколько раз ему был сопоставлен данный тег, и на основе этих частот вычисляется условная вероятность тега (с использованием сглаживания Лапласа): 
$$P(tag|word) = \dfrac{Fr(tag, word) + 1}{Fr(word) + R(word)} $$
где $Fr(tag, word)$ - количество раз, когда данная словоформа $word$ встретилась с тегом (т.е. с данными грамматическими характеристиками) $tag$ в корпусе **OpenCorpora**, $Fr(word)$ - количество раз, когда встретилась данная словоформа (уже без учета тега), $R(word)$ - число выведенных разборов анализатора для нашего слова $word$.

Разборы сортируются по убыванию score, поэтому первый разбор `morph.parse('стали')[0]` наиболее вероятный. 

Оценки $P(tag|word)$ помогают улучшить разбор, но их недостаточно для надежного снятия неоднозначности, как минимум по следующим причинам:
*   то, как нужно разбирать слово, зависит от соседних слов, а **pymorphy2** работает только на уровне отдельных слов;
*   условная вероятность $P(tag|word)$ оценена на основе сбалансированного набора текстов; в специализированных текстах вероятности могут быть другими - например, возможно, что в металлургических текстах $P(NOUN|стали) \gt P(VERB|стали)$;
*   в OpenCorpora у большинства слов неоднозначность пока не снята; 

 ### Разбор несловарных слов


In [15]:
morph.parse('наибольший')

[Parse(word='наибольший', tag=OpencorporaTag('ADJF,Supr,Qual masc,sing,nomn'), normal_form='больший', score=0.5, methods_stack=((DictionaryAnalyzer(), 'наибольший', 531, 27),)),
 Parse(word='наибольший', tag=OpencorporaTag('ADJF,Supr,Qual inan,masc,sing,accs'), normal_form='больший', score=0.5, methods_stack=((DictionaryAnalyzer(), 'наибольший', 531, 31),))]

### Оценка производительности

###Токенизация

In [30]:
text = "Я учусь в физико-математическом институте. Меня зовут К.С.Астанова."

In [31]:
!pip install tensorflow-text spacy==3.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import nltk
import spacy

In [33]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [34]:
from nltk.corpus.reader.tagged import word_tokenize
words = word_tokenize(text)
words

['Я',
 'учусь',
 'в',
 'физико-математическом',
 'институте',
 '.',
 'Меня',
 'зовут',
 'К.С.Астанова',
 '.']

In [35]:
for w in words:
  print(morph.parse(w)[0])

Parse(word='я', tag=OpencorporaTag('NPRO,1per sing,nomn'), normal_form='я', score=0.2941176470588234, methods_stack=((DictionaryAnalyzer(), 'я', 3246, 0),))
Parse(word='учусь', tag=OpencorporaTag('VERB,impf,intr sing,1per,pres,indc'), normal_form='учиться', score=1.0, methods_stack=((DictionaryAnalyzer(), 'учусь', 3102, 1),))
Parse(word='в', tag=OpencorporaTag('PREP'), normal_form='в', score=0.999327, methods_stack=((DictionaryAnalyzer(), 'в', 393, 0),))
Parse(word='физико-математическом', tag=OpencorporaTag('ADJF masc,sing,loct'), normal_form='физико-математический', score=0.5, methods_stack=((DictionaryAnalyzer(), 'физико-математическом', 16, 6),))
Parse(word='институте', tag=OpencorporaTag('NOUN,inan,masc sing,loct'), normal_form='институт', score=1.0, methods_stack=((DictionaryAnalyzer(), 'институте', 34, 5),))
Parse(word='.', tag=OpencorporaTag('PNCT'), normal_form='.', score=1.0, methods_stack=((PunctuationAnalyzer(score=0.9), '.'),))
Parse(word='меня', tag=OpencorporaTag('NPRO,1

In [36]:
!python -m spacy download ru_core_news_sm

2023-04-27 20:49:45.180018: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.3.0/ru_core_news_sm-3.3.0-py3-none-any.whl#egg=ru_core_news_sm==3.3.0 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 11.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [42]:
nlp = spacy.load('ru_core_news_sm')
doc = nlp(text)
tokens = []
for token in doc:
  tokens.append(str(token))
#tokens
for token in tokens:
  print(morph.parse(token)[0])

Parse(word='я', tag=OpencorporaTag('NPRO,1per sing,nomn'), normal_form='я', score=0.2941176470588234, methods_stack=((DictionaryAnalyzer(), 'я', 3246, 0),))
Parse(word='учусь', tag=OpencorporaTag('VERB,impf,intr sing,1per,pres,indc'), normal_form='учиться', score=1.0, methods_stack=((DictionaryAnalyzer(), 'учусь', 3102, 1),))
Parse(word='в', tag=OpencorporaTag('PREP'), normal_form='в', score=0.999327, methods_stack=((DictionaryAnalyzer(), 'в', 393, 0),))
Parse(word='физико', tag=OpencorporaTag('NOUN,anim,masc,Sgtm,Fixd,Name sing,nomn'), normal_form='физико', score=0.1375, methods_stack=((FakeDictionary(), 'физико', 25, 0), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'ико')))
Parse(word='-', tag=OpencorporaTag('PNCT'), normal_form='-', score=1.0, methods_stack=((PunctuationAnalyzer(score=0.9), '-'),))
Parse(word='математическом', tag=OpencorporaTag('ADJF masc,sing,loct'), normal_form='математический', score=0.5, methods_stack=((DictionaryAnalyzer(), 'математическом', 